<a href="https://colab.research.google.com/github/dr-song-summer-project/ChatBot_NLP/blob/main/Textrank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.2 MB/s 
     |████████████████████████████████| 86 kB 6.0 MB/s 
     |████████████████████████████████| 448 kB 40.8 MB/s 
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


Rank

In [2]:
import numpy as np
from sklearn.preprocessing import normalize

def pagerank(x, df=0.85, max_iter=30, bias=None):
    """
    Arguments
    ---------
    x : scipy.sparse.csr_matrix
        shape = (n vertex, n vertex)
    df : float
        Damping factor, 0 < df < 1
    max_iter : int
        Maximum number of iteration
    bias : numpy.ndarray or None
        If None, equal bias
    Returns
    -------
    R : numpy.ndarray
        PageRank vector. shape = (n vertex, 1)
    """

    assert 0 < df < 1

    # initialize
    A = normalize(x, axis=0, norm='l1')
    R = np.ones(A.shape[0]).reshape(-1,1)

    # check bias
    if bias is None:
        bias = (1 - df) * np.ones(A.shape[0]).reshape(-1,1)
    else:
        bias = bias.reshape(-1,1)
        bias = A.shape[0] * bias / bias.sum()
        assert bias.shape[0] == A.shape[0]
        bias = (1 - df) * bias

    # iteration
    for _ in range(max_iter):
        R = df * (A * R) + bias

    return R

sentence


In [3]:
from collections import Counter
import math
import numpy as np
import scipy as sp
from scipy.sparse import csr_matrix
from sklearn.metrics import pairwise_distances


def sent_graph(sents, tokenize=None, min_count=2, min_sim=0.3,
    similarity=None, vocab_to_idx=None, verbose=False):
    """
    Arguments
    ---------
    sents : list of str
        Sentence list
    tokenize : callable
        tokenize(sent) return list of str
    min_count : int
        Minimum term frequency
    min_sim : float
        Minimum similarity between sentences
    similarity : callable or str
        similarity(s1, s2) returns float
        s1 and s2 are list of str.
        available similarity = [callable, 'cosine', 'textrank']
    vocab_to_idx : dict
        Vocabulary to index mapper.
        If None, this function scan vocabulary first.
    verbose : Boolean
        If True, verbose mode on
    Returns
    -------
    sentence similarity graph : scipy.sparse.csr_matrix
        shape = (n sents, n sents)
    """

    if vocab_to_idx is None:
        idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    else:
        idx_to_vocab = [vocab for vocab, _ in sorted(vocab_to_idx.items(), key=lambda x:x[1])]

    x = vectorize_sents(sents, tokenize, vocab_to_idx)
    if similarity == 'cosine':
        x = numpy_cosine_similarity_matrix(x, min_sim, verbose, batch_size=1000)
    else:
        x = numpy_textrank_similarity_matrix(x, min_sim, verbose, batch_size=1000)
    return x

def vectorize_sents(sents, tokenize, vocab_to_idx):
    rows, cols, data = [], [], []
    for i, sent in enumerate(sents):
        counter = Counter(tokenize(sent))
        for token, count in counter.items():
            j = vocab_to_idx.get(token, -1)
            if j == -1:
                continue
            rows.append(i)
            cols.append(j)
            data.append(count)
    n_rows = len(sents)
    n_cols = len(vocab_to_idx)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

def numpy_cosine_similarity_matrix(x, min_sim=0.3, verbose=True, batch_size=1000):
    n_rows = x.shape[0]
    mat = []
    for bidx in range(math.ceil(n_rows / batch_size)):
        b = int(bidx * batch_size)
        e = min(n_rows, int((bidx+1) * batch_size))
        psim = 1 - pairwise_distances(x[b:e], x, metric='cosine')
        rows, cols = np.where(psim >= min_sim)
        data = psim[rows, cols]
        mat.append(csr_matrix((data, (rows, cols)), shape=(e-b, n_rows)))
        if verbose:
            print('\rcalculating cosine sentence similarity {} / {}'.format(b, n_rows), end='')
    mat = sp.sparse.vstack(mat)
    if verbose:
        print('\rcalculating cosine sentence similarity was done with {} sents'.format(n_rows))
    return mat

def numpy_textrank_similarity_matrix(x, min_sim=0.3, verbose=True, min_length=1, batch_size=1000):
    n_rows, n_cols = x.shape

    # Boolean matrix
    rows, cols = x.nonzero()
    data = np.ones(rows.shape[0])
    z = csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

    # Inverse sentence length
    size = np.asarray(x.sum(axis=1)).reshape(-1)
    size[np.where(size <= min_length)] = 10000
    size = np.log(size)

    mat = []
    for bidx in range(math.ceil(n_rows / batch_size)):

        # slicing
        b = int(bidx * batch_size)
        e = min(n_rows, int((bidx+1) * batch_size))

        # dot product
        inner = z[b:e,:] * z.transpose()

        # sentence len[i,j] = size[i] + size[j]
        norm = size[b:e].reshape(-1,1) + size.reshape(1,-1)
        norm = norm ** (-1)
        norm[np.where(norm == np.inf)] = 0

        # normalize
        sim = inner.multiply(norm).tocsr()
        rows, cols = (sim >= min_sim).nonzero()
        data = np.asarray(sim[rows, cols]).reshape(-1)

        # append
        mat.append(csr_matrix((data, (rows, cols)), shape=(e-b, n_rows)))

        if verbose:
            print('\rcalculating textrank sentence similarity {} / {}'.format(b, n_rows), end='')

    mat = sp.sparse.vstack(mat)
    if verbose:
        print('\rcalculating textrank sentence similarity was done with {} sents'.format(n_rows))

    return mat

def graph_with_python_sim(tokens, verbose, similarity, min_sim):
    if similarity == 'cosine':
        similarity = cosine_sent_sim
    elif callable(similarity):
        similarity = similarity
    else:
        similarity = textrank_sent_sim

    rows, cols, data = [], [], []
    n_sents = len(tokens)
    for i, tokens_i in enumerate(tokens):
        if verbose and i % 1000 == 0:
            print('\rconstructing sentence graph {} / {} ...'.format(i, n_sents), end='')
        for j, tokens_j in enumerate(tokens):
            if i >= j:
                continue
            sim = similarity(tokens_i, tokens_j)
            if sim < min_sim:
                continue
            rows.append(i)
            cols.append(j)
            data.append(sim)
    if verbose:
        print('\rconstructing sentence graph was constructed from {} sents'.format(n_sents))
    return csr_matrix((data, (rows, cols)), shape=(n_sents, n_sents))

def textrank_sent_sim(s1, s2):
    """
    Arguments
    ---------
    s1, s2 : list of str
        Tokenized sentences
    Returns
    -------
    Sentence similarity : float
        Non-negative number
    """
    n1 = len(s1)
    n2 = len(s2)
    if (n1 <= 1) or (n2 <= 1):
        return 0
    common = len(set(s1).intersection(set(s2)))
    base = math.log(n1) + math.log(n2)
    return common / base

def cosine_sent_sim(s1, s2):
    """
    Arguments
    ---------
    s1, s2 : list of str
        Tokenized sentences
    Returns
    -------
    Sentence similarity : float
        Non-negative number
    """
    if (not s1) or (not s2):
        return 0

    s1 = Counter(s1)
    s2 = Counter(s2)
    norm1 = math.sqrt(sum(v ** 2 for v in s1.values()))
    norm2 = math.sqrt(sum(v ** 2 for v in s2.values()))
    prod = 0
    for k, v in s1.items():
        prod += v * s2.get(k, 0)
    return prod / (norm1 * norm2)

summarizer

In [4]:

class KeywordSummarizer:
    """
    Arguments
    ---------
    sents : list of str
        Sentence list
    tokenize : callable
        Tokenize function: tokenize(str) = list of str
    min_count : int
        Minumum frequency of words will be used to construct sentence graph
    window : int
        Word cooccurrence window size. Default is -1.
        '-1' means there is cooccurrence between two words if the words occur in a sentence
    min_cooccurrence : int
        Minimum cooccurrence frequency of two words
    vocab_to_idx : dict or None
        Vocabulary to index mapper
    df : float
        PageRank damping factor
    max_iter : int
        Number of PageRank iterations
    verbose : Boolean
        If True, it shows training progress
    """
    def __init__(self, sents=None, tokenize=None, min_count=2,
        window=-1, min_cooccurrence=2, vocab_to_idx=None,
        df=0.85, max_iter=30, verbose=False):

        self.tokenize = tokenize
        self.min_count = min_count
        self.window = window
        self.min_cooccurrence = min_cooccurrence
        self.vocab_to_idx = vocab_to_idx
        self.df = df
        self.max_iter = max_iter
        self.verbose = verbose

        if sents is not None:
            self.train_textrank(sents)

    def train_textrank(self, sents, bias=None):
        """
        Arguments
        ---------
        sents : list of str
            Sentence list
        bias : None or numpy.ndarray
            PageRank bias term
        Returns
        -------
        None
        """

        g, self.idx_to_vocab = word_graph(sents,
            self.tokenize, self.min_count,self.window,
            self.min_cooccurrence, self.vocab_to_idx, self.verbose)
        self.R = pagerank(g, self.df, self.max_iter, bias).reshape(-1)
        if self.verbose:
            print('trained TextRank. n words = {}'.format(self.R.shape[0]))

    def keywords(self, topk=30):
        """
        Arguments
        ---------
        topk : int
            Number of keywords selected from TextRank
        Returns
        -------
        keywords : list of tuple
            Each tuple stands for (word, rank)
        """
        if not hasattr(self, 'R'):
            raise RuntimeError('Train textrank first or use summarize function')
        idxs = self.R.argsort()[-topk:]
        keywords = [(self.idx_to_vocab[idx], self.R[idx]) for idx in reversed(idxs)]
        return keywords

    def summarize(self, sents, topk=30):
        """
        Arguments
        ---------
        sents : list of str
            Sentence list
        topk : int
            Number of keywords selected from TextRank
        Returns
        -------
        keywords : list of tuple
            Each tuple stands for (word, rank)
        """
        self.train_textrank(sents)
        return self.keywords(topk)


class KeysentenceSummarizer:
    """
    Arguments
    ---------
    sents : list of str
        Sentence list
    tokenize : callable
        Tokenize function: tokenize(str) = list of str
    min_count : int
        Minumum frequency of words will be used to construct sentence graph
    min_sim : float
        Minimum similarity between sentences in sentence graph
    similarity : str
        available similarity = ['cosine', 'textrank']
    vocab_to_idx : dict or None
        Vocabulary to index mapper
    df : float
        PageRank damping factor
    max_iter : int
        Number of PageRank iterations
    verbose : Boolean
        If True, it shows training progress
    """
    def __init__(self, sents=None, tokenize=None, min_count=2,
        min_sim=0.3, similarity=None, vocab_to_idx=None,
        df=0.85, max_iter=30, verbose=False):

        self.tokenize = tokenize
        self.min_count = min_count
        self.min_sim = min_sim
        self.similarity = similarity
        self.vocab_to_idx = vocab_to_idx
        self.df = df
        self.max_iter = max_iter
        self.verbose = verbose

        if sents is not None:
            self.train_textrank(sents)

    def train_textrank(self, sents, bias=None):
        """
        Arguments
        ---------
        sents : list of str
            Sentence list
        bias : None or numpy.ndarray
            PageRank bias term
            Shape must be (n_sents,)
        Returns
        -------
        None
        """
        g = sent_graph(sents, self.tokenize, self.min_count,
            self.min_sim, self.similarity, self.vocab_to_idx, self.verbose)
        self.R = pagerank(g, self.df, self.max_iter, bias).reshape(-1)
        if self.verbose:
            print('trained TextRank. n sentences = {}'.format(self.R.shape[0]))

    def summarize(self, sents, topk=30, bias=None):
        """
        Arguments
        ---------
        sents : list of str
            Sentence list
        topk : int
            Number of key-sentences to be selected.
        bias : None or numpy.ndarray
            PageRank bias term
            Shape must be (n_sents,)
        Returns
        -------
        keysents : list of tuple
            Each tuple stands for (sentence index, rank, sentence)
        Usage
        -----
            >>> from textrank import KeysentenceSummarizer
            >>> summarizer = KeysentenceSummarizer(tokenize = tokenizer, min_sim = 0.5)
            >>> keysents = summarizer.summarize(texts, topk=30)
        """
        n_sents = len(sents)
        if isinstance(bias, np.ndarray):
            if bias.shape != (n_sents,):
                raise ValueError('The shape of bias must be (n_sents,) but {}'.format(bias.shape))
        elif bias is not None:
            raise ValueError('The type of bias must be None or numpy.ndarray but the type is {}'.format(type(bias)))

        self.train_textrank(sents, bias)
        idxs = self.R.argsort()[-topk:]
        keysents = [(idx, self.R[idx], sents[idx]) for idx in reversed(idxs)]
        return keysents

utils

In [5]:
from collections import Counter
from scipy.sparse import csr_matrix
import numpy as np


def scan_vocabulary(sents, tokenize=None, min_count=2):
    """
    Arguments
    ---------
    sents : list of str
        Sentence list
    tokenize : callable
        tokenize(str) returns list of str
    min_count : int
        Minumum term frequency
    Returns
    -------
    idx_to_vocab : list of str
        Vocabulary list
    vocab_to_idx : dict
        Vocabulary to index mapper.
    """
    counter = Counter(w for sent in sents for w in tokenize(sent))
    counter = {w:c for w,c in counter.items() if c >= min_count}
    idx_to_vocab = [w for w, _ in sorted(counter.items(), key=lambda x:-x[1])]
    vocab_to_idx = {vocab:idx for idx, vocab in enumerate(idx_to_vocab)}
    return idx_to_vocab, vocab_to_idx

def tokenize_sents(sents, tokenize):
    """
    Arguments
    ---------
    sents : list of str
        Sentence list
    tokenize : callable
        tokenize(sent) returns list of str (word sequence)
    Returns피임
    -------
    tokenized sentence list : list of list of str
    """
    return [tokenize(sent) for sent in sents]

def vectorize(tokens, vocab_to_idx):
    """
    Arguments
    ---------
    tokens : list of list of str
        Tokenzed sentence list
    vocab_to_idx : dict
        Vocabulary to index mapper
    Returns
    -------
    sentence bow : scipy.sparse.csr_matrix
        shape = (n_sents, n_terms)
    """
    rows, cols, data = [], [], []
    for i, tokens_i in enumerate(tokens):
        for t, c in Counter(tokens_i).items():
            j = vocab_to_idx.get(t, -1)
            if j == -1:
                continue
            rows.append(i)
            cols.append(j)
            data.append(c)
    n_sents = len(tokens)
    n_terms = len(vocab_to_idx)
    x = csr_matrix((data, (rows, cols)), shape=(n_sents, n_terms))
    return x


word

In [6]:
from collections import defaultdict
from scipy.sparse import csr_matrix


def word_graph(sents, tokenize=None, min_count=2, window=2,
    min_cooccurrence=2, vocab_to_idx=None, verbose=False):
    """
    Arguments
    ---------
    sents : list of str
        Sentence list
    tokenize : callable
        tokenize(str) returns list of str
    min_count : int
        Minumum term frequency
    window : int
        Co-occurrence window size
    min_cooccurrence : int
        Minimum cooccurrence frequency
    vocab_to_idx : dict
        Vocabulary to index mapper.
        If None, this function scan vocabulary first.
    verbose : Boolean
        If True, verbose mode on
    Returns
    -------
    co-occurrence word graph : scipy.sparse.csr_matrix
    idx_to_vocab : list of str
        Word list corresponding row and column
    """
    if vocab_to_idx is None:
        idx_to_vocab, vocab_to_idx = scan_vocabulary(sents, tokenize, min_count)
    else:
        idx_to_vocab = [vocab for vocab, _ in sorted(vocab_to_idx.items(), key=lambda x:x[1])]

    tokens = tokenize_sents(sents, tokenize)
    g = cooccurrence(tokens, vocab_to_idx, window, min_cooccurrence, verbose)
    return g, idx_to_vocab

def cooccurrence(tokens, vocab_to_idx, window=2, min_cooccurrence=2, verbose=False):
    """
    Arguments
    ---------
    tokens : list of list of str
        Tokenized sentence list
    vocab_to_idx : dict
        Vocabulary to index mapper
    window : int
        Co-occurrence window size
    min_cooccurrence : int
        Minimum cooccurrence frequency
    verbose : Boolean
        If True, verbose mode on
    Returns
    -------
    co-occurrence matrix : scipy.sparse.csr_matrix
        shape = (n_vocabs, n_vocabs)
    """
    counter = defaultdict(int)
    for s, tokens_i in enumerate(tokens):
        if verbose and s % 1000 == 0:
            print('\rword cooccurrence counting {}'.format(s), end='')
        vocabs = [vocab_to_idx[w] for w in tokens_i if w in vocab_to_idx]
        n = len(vocabs)
        for i, v in enumerate(vocabs):
            if window <= 0:
                b, e = 0, n
            else:
                b = max(0, i - window)
                e = min(i + window, n)
            for j in range(b, e):
                if i == j:
                    continue
                counter[(v, vocabs[j])] += 1
                counter[(vocabs[j], v)] += 1
    counter = {k:v for k,v in counter.items() if v >= min_cooccurrence}
    n_vocabs = len(vocab_to_idx)
    if verbose:
        print('\rword cooccurrence counting from {} sents was done'.format(s+1))
    return dict_to_mat(counter, n_vocabs, n_vocabs)

def dict_to_mat(d, n_rows, n_cols):
    """
    Arguments
    ---------
    d : dict
        key : (i,j) tuple
        value : float value
    Returns
    -------
    scipy.sparse.csr_matrix
    """
    rows, cols, data = [], [], []
    for (i, j), v in d.items():
        rows.append(i)
        cols.append(j)
        data.append(v)
    return csr_matrix((data, (rows, cols)), shape=(n_rows, n_cols))

In [7]:
from konlpy.tag import Komoran

komoran = Komoran()
def komoran_tokenize(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

text입력

In [8]:
from google.colab import files

uploaded = files.upload()

Saving keyword.xlsx to keyword.xlsx


In [29]:
import pandas as pd

# path = '/content/newxlsx.xlsx'
# df = pd.read_excel(path)
# data = pd.DataFrame.to_numpy(df)

# test = []
# for x in data:
#   test.append(x[0])


# path = '/content/clustered_8.xlsx'  
# df = pd.read_excel(path)
# data = pd.DataFrame.to_numpy(df)


# test = []
# for x in data:
#   if x[2] == 2:
#     test.append(x[1])

path = '/content/keyword.xlsx'
df = pd.read_excel(path)
data = pd.DataFrame.to_numpy(df)


test = [[] for _ in range(19)]
idx = 0
for x in data:
  # if x[2] == 1:
  #   test[0].append(x[0])
  if x[2] == 0:
    continue
  idx = x[2]
  test[idx-1].append(x[0])

In [32]:
print(test[2])

['\xa01월 27일 생리 시작일이며 피임약 야즈를 도 중에 끊고 27일부터 다시 먹었습니다. 그런데 아직까지도 생리가 끝나지 않아서 질문 드려요. 약을 다시 끊어야 하나요? 예전에 먹을 때는 이런 적 없었습니다.', '\xa02주 전에 생리 시작하면서 피임약 복용을 함께 시작했는데요, 지난주 토요일에 제가 다리를 다치면서 정신이 없어서인지 일, 월, 화 3일치 약 복용을 잊었습니다. 밤 10시경에 항상 먹었어서 일단 화요일 밤에 이틀치를 먹고 수요일 아침에 화요일 분을, 수요일 밤 10시부터 다시 원래대로 복용하고 있는데요. 약 복용을 잊고 있다가 생각나게 된 계기가 몸의 변화 때문입니다. 생리가 끝난 지 일주일밖에 안 됐는데 약을 안 먹는 며칠 동안 출혈이 조금씩 나 고 생리 시작할 때 만큼은 아니지만 배가 살살 아프더라구요. 다시 피임약 복용을 하고 있는데 출혈은 멈췄지만 가슴이 부 풀어 오르고 속이 메슥거리는 등 생리 전중 증상이 나타납니다. 계속 피임약을 복용해야 할까요? 왜 이러는지 걱정되네요.. br', '\xa03일 후인 13일 날 피임약 복용이 끝나는 복용 3주차입니다 일주일 뒤 일정이 생겨서 생리를 미루고 싶은데 어떻게 해야 할까요? 도와 주세요 ㅜㅜ 20일까지 생리를 미루고 싶습니다! 1. 피임약을 추가 복용해서 미뤄야 하나요? 2. 피임약을 3일 기간 동안 끊고 지금 생리 후에 새로 복용하면 될까요? 언제 복용하면 될까요? br', '\xa0뚱뚱하고 40대입니다. 생리를 3달 이상 안한 후 하기에 생리인 줄 알았으나 출혈이 멈추지 않아 피임약 복용하고 있읍니다. 피임약을 3주 먹고 1주 쉬라고 했으나 출혈양이 적어지기는 했으나 멈추지 않아 쉬지 않고 4주째 먹고 있읍니다. 피양이 다시 조금 증가 한 것 같은데 피임약을 쉬지 않고 먹으면 어떻게 되나요?', '\xa0제가 4월부터 피임약을 복용하고 휴약기에 정상적으로 생리를 하고 5월에도 날짜를 맞춰서 복용을 했는데 몸이 아파서 1주일 정도 복용 후에 중단을 하고 지금까지 복용을 하지 않았

In [37]:
# summarizer = KeysentenceSummarizer(
#     tokenize = komoran_tokenize,
#     min_sim = 0.5,
#     verbose = False
# )

# keysents = summarizer.summarize(test, topk=5)
# # print(keysents)
# for x in keysents:
#   print(x)

summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenize,
    min_sim = 0.5,
    verbose = True
)

keysents = summarizer.summarize(test[6], topk=5)
for sent_idx, rank, sent in keysents:  
  print(f'{sent_idx} : {rank} :: {sent}')

calculating textrank sentence similarity was done with 35 sents
trained TextRank. n sentences = 35
16 : 1.3416150679141858 ::  strng 질문1.strng 피임약 질문하기 전에요 3달 전부터 생리가 3일 정도만 하고 끗 나는데 왜 그런 거죠 ㅠ원래 5~7일 정도였는데 ㅠㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡstrng 질문2.strng 피임약 이번에 처음 복용했어요 생리 시작하는 날 바로 먹고 처음 2주 정도는 피임 잘 안 된다 해서 관계 안 하고 2주 뒤에 했구요 제 때 제때 시간 맞춰서 잘 먹었는데 7일 휴약기간 동안 생리가 나와야 하는데 안 나오네요 ㅠ 혹시 피임잘 안되서 그런 건가 하고 임신테스트 기사서 검사해봤는데 임신 아니더라구요 근데 제가 생리주기가 불순하기도 하고 쫌길 기도한 데 34일~40일 정도 해여 6월-22일 생리 7월-25일 생리 이렇거든요ㅜ 오늘이 17일 이니까 기간상 으로 봤을 때는 생리 시작할라면 약 10일 정도 지나야 되요제가 질문하고 싶은 거는 요ㅠ 1. 피임약 봉용 후 7일 휴약기간 동안 생리를 안해도 8일째 되는 날 약을 다시 복용해도 되나요? 2. 아니면 생리나올 때까지 기다렸다가 복용해야 되나요? 3. 만 약 8일째 되는 날 복용한다면 이번 달 생리는 언제나오는 거죠 ㅠㅠ 4. 그냥 8일째 되는 날 피임약 안 먹고 남자친구 보고 콘돔 사용하라고 할까요?.. 1년 반 정도 만나면서 관계를 가졌는데 계속 콘돔을 사용하다가 좀 불편하고 혹시 몰라서 피임약 한번 복용해본 거거든요 br
11 : 1.2693250300861916 ::  피임약을 2월 8일 서부터 먹고 있습니다. 생리주기는 불규칙합니다. (피임약 규칙적으로 먹었을 땐 30일이었는데 저번 달은 40일이 더군요.)만 약 피임약을 하루 정도 빼고 약 복용하면 피임은 되지 않고 생리만 안 하게 되잖아요? 그럼 배란 일은 대략 언 제쯤 되나요? 제가

In [ ]:
docs = ['list of str form', 'sentence list']

keyword_extractor = KeywordSummarizer(
    tokenize = komoran_tokenize,      # YOUR TOKENIZER
    window = -1,
    verbose = False
)

keywords = keyword_extractor.summarize(test, topk=30)
for word, rank in keywords:
    print(word, rank)

In [36]:
for i in range(19):
  print(f'=================={i+1}번째====================')
  keywords = keyword_extractor.summarize(test[i], topk=10)
  for word, rank in keywords:
      print(word, rank)

==================1번째====================
먹/VV 2.874699472361487
되/VV 1.896231742437004
피임약/NNP 1.877587457187372
비타민/NNP 1.387363707888139
정/NNP 1.3052228591371025
있/VV 1.1926259825938477
경구/NNG 1.1470127162051185
피임/NNP 1.1130538799875076
하/VV 1.0632976487754808
말/NNG 0.8962953109808395
==================2번째====================
하/VV 16.707718849379663
일/NNB 13.64215235877169
생리/NNG 12.889681041760678
피임약/NNP 10.877405467525534
먹/VV 9.378196340773256
복용/NNG 9.349282790816225
후/NNG 7.700320598492748
관계/NNG 7.386592316703711
임신/NNP 7.268656747552454
있/VV 6.794992799841826
==================3번째====================
생리/NNG 15.360384761252103
하/VV 14.8422122075201
복용/NNG 10.850170764676163
피임약/NNP 9.618958225308003
일/NNB 8.14524089085127
있/VV 4.769472615091315
먹/VV 4.734686651836285
약/NNG 4.413240739760327
시작/NNG 3.8623017890965694
후/NNG 3.3724686834541964
==================4번째====================
먹/VV 13.304344562533124
복용/NNG 10.72449752207666
일/NNB 9.017863894554894
생리/NNG 8.969702498538